In [ ]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [ ]:
batch_size = 32 
img_height = 224
img_width = 224 

In [ ]:
validation_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=None, seed=42)

In [ ]:
training_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)

## if Xception:

In [ ]:
def preprocess(image, label):
    final_image = keras.applications.xception.preprocess_input(image)
    return final_image, label

In [ ]:
training_s = training_ds.map(preprocess).batch(batch_size).prefetch(1)
validation_s = validation_ds.map(preprocess).batch(batch_size).prefetch(1)

## else:

In [ ]:
training_s = training_ds.batch(batch_size).prefetch(1)
validation_s = validation_ds.batch(batch_size).prefetch(1)

## OR

In [ ]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

## Base model & Weight

In [ ]:
# base_model = keras.applications.Model
# Model(input_shape=(224,224,3), include_top = False, weights=___)

In [ ]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

## if weight = 'imagenet':

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Adam default lr = 0.001, SGD default lr = 0.01

In [ ]:
model.fit(training_s, validation_data=validation_s, epochs=3)

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

## if weight = None: skip the above

In [ ]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

## learning_rate should be smaller

In [ ]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

## patience = 2 & epochs=5 // patience = 5 & epochs=20 or more

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_ps, validation_data=validation_ps, epochs=10, callbacks=[es])

In [ ]:
model.evaluate(training_ps)

In [ ]:
model.evaluate(validation_ps)

In [ ]:
model.save("!!.h5")